In [1]:
import subprocess
import os

import pandas as pd

import requests
from bs4 import BeautifulSoup

import json


pd.set_option('display.max_rows', 500)

# Data Understanding

1. RKI, webscrape (webscraping) https://www.rki.de/DE/Content/InfAZ/N/Neuartiges_Coronavirus/Fallzahlen.html
2. John Hopkins (GITHUB) https://github.com/CSSEGISandData/COVID-19.git
3. REST API services to retreive data https://npgeo-corona-npgeo-de.hub.arcgis.com/

# GITHUB csv data

git clone/pull https://github.com/CSSEGISandData/COVID-19.git

In [2]:

# git_pull = subprocess.Popen( "git pull" , 
#                      cwd = os.path.dirname( '../data/raw/COVID-19/' ), 
#                      shell = True, 
#                      stdout = subprocess.PIPE, 
#                      stderr = subprocess.PIPE )
# (out, error) = git_pull.communicate()


# print("Error : " + str(error)) 
# print("out : " + str(out))

In [3]:
data_path='../data/raw/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
pd_raw=pd.read_csv(data_path)

In [4]:
    pd_raw.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,6/17/22,6/18/22,6/19/22,6/20/22,6/21/22,6/22/22,6/23/22,6/24/22,6/25/22,6/26/22
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,181534,181574,181666,181725,181808,181912,181987,182033,182072,182149
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,277141,277141,277409,277444,277663,277940,278211,278504,278793,279077
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,265968,265971,265975,265985,265993,266006,266015,266025,266030,266038
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,...,43449,43449,43449,43449,43449,43774,43774,43774,43774,43774
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,...,99761,99761,99761,99761,99761,99761,99761,99761,99761,99761


# Webscraping



In [4]:
page = requests.get("https://www.rki.de/DE/Content/InfAZ/N/Neuartiges_Coronavirus/Fallzahlen.html")

In [5]:
soup = BeautifulSoup(page.content, 'html.parser')


In [6]:
html_table=soup.find('table') # find the table, attention this works if one table exists

In [7]:
all_rows=html_table.find_all('tr')

In [8]:
final_data_list=[]

In [9]:
for pos,rows in enumerate(all_rows):
   
    col_list=[each_col.get_text(strip=True) for each_col in rows.find_all('td')] #td for data element
    final_data_list.append(col_list)

print(final_data_list)

[[], [], ['Baden-Württem\xadberg', '5.009.839', '1.761', '8.171', '73,4', '18.985'], ['Bayern', '6.668.223', '2.665', '14.008', '106,3', '28.116'], ['Berlin', '1.418.625', '260', '2.043', '55,6', '5.446'], ['Branden\xadburg', '1.104.261', '246', '1.504', '59,3', '6.438'], ['Bremen', '301.225', '-52', '662', '97,9', '956'], ['Hamburg', '802.814', '146', '797', '43,0', '3.544'], ['Hessen', '2.887.109', '938', '6.306', '100,2', '12.179'], ['Meck\xadlenburg-Vor\xadpommern', '706.889', '211', '1.262', '78,3', '2.733'], ['Nieder\xadsachsen', '3.820.177', '1.625', '11.062', '137,8', '13.190'], ['Nord\xadrhein-West\xadfalen', '7.918.329', '3.715', '21.489', '119,9', '30.862'], ['Rhein\xadland-Pfalz', '1.742.709', '679', '3.403', '82,9', '6.833'], ['Saarland', '484.737', '178', '1.023', '104,1', '2.087'], ['Sachsen', '1.944.903', '342', '2.055', '50,8', '16.744'], ['Sachsen-Anhalt', '954.211', '262', '1.434', '66,1', '6.236'], ['Schles\xadwig-Holstein', '1.167.605', '277', '1.525', '52,2', '3.4

In [10]:
pd_daily_status=pd.DataFrame(final_data_list).dropna().rename(columns={0:'states',
                                                       1:'cases',
                                                       2:'changes',
                                                       3:'cases_per_100k',
                                                       4:'fatal',
                                                       5:'comment'})

In [11]:
pd_daily_status.head()

,states,cases,changes,cases_per_100k,fatal,comment
2,Baden-Württem­berg,5.009.839,1.761,8.171,"73,4",18.985
3,Bayern,6.668.223,2.665,14.008,"106,3",28.116
4,Berlin,1.418.625,260,2.043,"55,6",5.446
5,Branden­burg,1.104.261,246,1.504,"59,3",6.438
6,Bremen,301.225,-52,662,"97,9",956


# REST API calls

In [12]:
## data request for Germany
data=requests.get('https://services7.arcgis.com/mOBPykOjAyBO2ZKk/arcgis/rest/services/Coronaf%C3%A4lle_in_den_Bundesl%C3%A4ndern/FeatureServer/0/query?where=1%3D1&outFields=*&outSR=4326&f=json')


In [13]:
json_object=json.loads(data.content) 


In [14]:
type(json_object)

dict

In [15]:
json_object.keys()

dict_keys(['objectIdFieldName', 'uniqueIdField', 'globalIdFieldName', 'geometryProperties', 'geometryType', 'spatialReference', 'fields', 'features'])

In [16]:
full_list=[]
for pos,each_dict in enumerate (json_object['features'][:]):
    full_list.append(each_dict['attributes'])
    

In [17]:
pd_full_list=pd.DataFrame(full_list)
pd_full_list.head()

,OBJECTID_1,LAN_ew_AGS,LAN_ew_GEN,LAN_ew_BEZ,LAN_ew_EWZ,OBJECTID,Fallzahl,Aktualisierung,AGS_TXT,GlobalID,faelle_100000_EW,Shape__Area,Shape__Length,Death,cases7_bl_per_100k,cases7_bl,death7_bl,cases7_bl_per_100k_txt,AdmUnitId
0,1,01,Schleswig-Holstein,Land,2922005,15,1167605,1675378800000,01,fc5ba936-c95c-432c-8a33-9eb2f30b660f,39959.034978,4.573731e+10,2.881496e+06,3443,52.190191,1525,2,"52,2",1
1,2,02,Hamburg,Freie und Hansestadt,1853935,6,802814,1675378800000,02,0f3e860c-5181-4d3f-a421-1d51f50315ea,43303.244181,2.089396e+09,4.188002e+05,3544,42.989641,797,2,"43,0",2
2,3,03,Niedersachsen,Land,8027031,9,3820177,1675378800000,03,3fd77024-c29b-4843-9be8-682ad48e60c9,47591.407084,1.299836e+11,4.008988e+06,13190,137.809359,11062,4,"137,8",3
3,4,04,Bremen,Freie Hansestadt,676463,5,301225,1675378800000,04,4132268b-54de-4327-ac1e-760e915112f1,44529.412547,1.119157e+09,3.357177e+05,956,97.861967,662,1,"97,9",4
4,5,05,Nordrhein-Westfalen,Land,17924591,10,7918329,1675378800000,05,561d658f-3ee5-46e3-bc95-3528c6558ab9,44175.786215,8.782936e+10,2.648673e+06,30862,119.885581,21489,8,"119,9",5


In [18]:
pd_full_list.to_csv('../data/raw/COVID-19/csse_covid_19_data/csse_covid_19_time_series.csv',sep=';')

In [19]:
pd_full_list.shape[0]

16